In [1]:
import pandas as pd
import numpy as np

### Reading files

In [2]:
deliveries_df = pd.read_csv("data/deliveries_formatted.csv")
matches_df = pd.read_csv("data/matches_formatted.csv")
ball_matches_df = pd.read_csv("data/ball_matches.csv")

C:\Users\n.sundaramahalingam\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

# History Analysis

### individual batsman history

In [5]:
batsman = 'DA Warner'

In [8]:
batsman_history = ball_matches_df.groupby(['season','match_id','batsman']).agg(batsman_score = ('batsman_runs','sum')).reset_index()

In [10]:
# batsman_history[batsman_history.batsman == batsman]

### individual bowler history

In [18]:
bowler = 'SL Malinga'

In [15]:
bowler_history = ball_matches_df[ball_matches_df.dismissal_kind.isin(['caught', 'bowled', 'lbw', 'caught and bowled', 'hit wicket'])].groupby(['season','match_id','bowler']).agg(bowler_wicket = ('dismissal_kind','count')).reset_index()

In [20]:
# bowler_history[bowler_history.bowler == bowler]

### Team head to head

In [61]:
teams = ['CSK', 'KXIP']

In [62]:
team_head_to_head = ball_matches_df[(((ball_matches_df.team1==teams[0]) & (ball_matches_df.team2==teams[1])) | ((ball_matches_df.team1==teams[1]) & (ball_matches_df.team2==teams[0]))) ]

In [63]:
columns = ['id', 'season', 'date', 'toss_winner',
       'toss_decision', 'result', 'winner', 'win_by_runs',
       'win_by_wickets', 'player_of_match', 'venue','inning', 'batting_team', 'bowling_team', 'over',
       'ball', 'batsman', 'bowler', 'is_super_over',
       'wide_runs', 'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder']

In [64]:
team_head_to_head_rel = team_head_to_head[columns] 

In [65]:
team_head_to_head_winning = team_head_to_head_rel[team_head_to_head_rel.result == 'normal'].groupby(['id','winner']).agg(no_of_winning = ('winner','nunique')).reset_index()

In [71]:
# team_head_to_head_winning.groupby('winner').agg({'no_of_winning':'count'})

In [69]:
team_head_to_head_summary = team_head_to_head_rel.groupby(['id','result']).agg(summary = ('result','nunique')).reset_index()

In [72]:
# team_head_to_head_summary.groupby('result').agg({'summary':'count'})

### overall IPL matches

In [97]:
ipl_matches = ball_matches_df[ball_matches_df.inning.isin([1,2])].groupby(['id', 'season','inning','batting_team']).agg(score=('total_runs','sum'), wickets =('player_dismissed','count')).reset_index()

In [106]:
ipl_matches_pivoted = ipl_matches.pivot_table(index=['id','season'],columns=['inning'], values=['batting_team','score','wickets'], aggfunc='first').reset_index()

In [126]:
# ipl_matches_pivoted.head()

In [155]:
ipl_history = pd.DataFrame(ipl_matches_pivoted.to_records())

In [156]:
# ipl_history[ipl_history.match_id==301]

In [157]:
fillna_values = {"('batting_team', 1)": 'Not Available', "('batting_team', 2)": 'Not Available', "('score', 1)": 0, 
                 "('score', 2)": 0, "('wickets', 1)":0 , "('wickets', 2)":0}

In [158]:
ipl_history.fillna(value=fillna_values, inplace=True)

In [159]:
# ipl_history[ipl_history.match_id==301]

In [160]:
ipl_history['match_id'] = ipl_history["('id', '')"]
ipl_history['first_innings_team'] = ipl_history["('batting_team', 1)"]
ipl_history['first_bat_score']=ipl_history["('score', 1)"].astype(int).astype(str)+"\ "+ ipl_history["('wickets', 1)"].astype(int).astype(str)

ipl_history['second_innings_team'] = ipl_history["('batting_team', 2)"]
ipl_history['second_bat_score']=ipl_history["('score', 2)"].astype(int).astype(str)+"\ "+ ipl_history["('wickets', 2)"].astype(int).astype(str)

In [161]:
ipl_history.drop(columns=['index', "('id', '')", "('season', '')", "('batting_team', 1)",
       "('batting_team', 2)", "('score', 1)", "('score', 2)", "('wickets', 1)",
       "('wickets', 2)"], inplace=True)

In [166]:
# ipl_history

In [167]:
ipl_history_all = pd.merge(ipl_history, matches_df, left_on ='match_id', right_on='id')

In [169]:
# ipl_history_all

In [171]:
def win_type(item):
    if item==0:
        return 'Chasing'
    else:
        return 'Batting'

In [172]:
ipl_history_all['win_by_chase/bat']=list(map(win_type,ipl_history_all['win_by_runs']))

In [180]:
ipl_history_win = ipl_history_all.groupby(['venue','win_by_chase/bat']).agg(win_count = ('win_by_chase/bat','count')).reset_index()

In [184]:
# ipl_history_win.head()

In [183]:
# ipl_history_win.pivot_table(index='venue',columns='win_by_chase/bat',values='win_count').reset_index()